In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import time
import random

from heapq import nlargest

from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance
#plot
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from matplotlib.axes import Axes

In [2]:
X_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_x_resample3000_boundarysel_50pergroup.csv")
y_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_y_resample3000_boundarysel_50pergroup.csv")
X_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\scaled_validation_feature.csv")
y_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\validation_label.csv")

In [3]:
X_train = X_train.drop(["predict_cluster"],axis=1)
y_train = y_train.drop(["predict_cluster"],axis=1)
X_val = X_val.drop(["Unnamed: 0"],axis=1)
y_val = y_val.drop(["Unnamed: 0"],axis=1)

In [35]:
def knn_predict(X_val,X_train,y_train,k_value):
    start=time.time()
    y_pred=[]
    #y_proba_1=[]
    #y_proba_2=[]
    cov_train=np.matrix(X_train.cov())
    print("KNN start")
    start=time.time()
    for test_index in range(len(X_val)):
        #if test_index%100==0:
        #    print("the X_val is %d and time is %5.1f minute"%(test_index,(time.time()-start)/60))
        good=0
        bad=0
        sample=np.array(X_val.iloc[test_index]).reshape(1,-1)
        m_dis=pd.Series((distance.cdist(X_train,sample,metric='euclidean',V=cov_train)).ravel())
        #.sort_values(ascending=True)#average computing time 0.12s
        #dis_set=m_dis[:k_value]# get the k nearest mahalanobis distance
        m_dis = np.array(m_dis)
        largest_index=np.argsort(m_dis)[-k_value:]
        #dis_set=nlargest(k_value, m_dis)
        for index in largest_index:
            if y_train.loc[index] == 1.0:
                good += 1#dis_set.loc[index] #added the weight to the vote
            else:
                bad += 1#dis_set.loc[index]
        if good > bad :
            y_pred.append(np.float64(1.0))
        else:
            y_pred.append(np.float64(2.0))
        #y_proba_1.append(good/(good+bad))
        #y_proba_2.append(bad/(good+bad))
    print("time usage:%5.1fminute"%((time.time()-start)/60))
    print("KNN STOP")
    y_pred=pd.Series(y_pred)
    return y_pred #,y_proba_1,y_proba_2

In [36]:
y_pred = knn_predict(X_val,X_train,y_train['label'],5)

KNN start
time usage: 38.9minute
KNN STOP


In [37]:
y_pred = {"label_pred":y_pred}
y_pred = pd.DataFrame(data=y_pred)

In [40]:
Precall,f1_score,BER,FP = cal_score (y_pred,y_val['label']) 

n11:..27601..n12:..7766..n21:..33606..n22:..1054
TP:0.11950113378684807
f1 score:0.0484820607175713
FP:0.5490548466678647
BER:0.7147768564405084


In [39]:
def cal_score (y_pred,y_val):
    n11 = 0
    n12 = 0
    n21 = 0
    n22 = 0
    y_pred_array= np.array(y_pred)
    y_val_array= np.array(y_val)
    for j in range(len(y_pred_array)):
        if (y_pred_array[j]==2)&(y_val_array[j]==2):
            n22 = n22+1
        elif (y_pred_array[j]==1)&(y_val_array[j]==2):
            n12 = n12 +1
        elif (y_pred_array[j]==2)&(y_val_array[j]==1):
            n21 = n21+1
        else:
            n11 = n11+1   
    '''
    ita = np.nditer (y_pred_array)
    itb = np.nditer (y_val_array)
    while not ita.finished:
        if (ita[0]==2)&(itb[0]==2):
            n22 = n22+1
        elif (ita[0]==1)&(itb[0]==2):
            n12 = n12+1
        elif (ita[0]==2)&(itb[0]==1):
            n21 = n21+1
        else:
            n11 = n11+1
        ita.iternext()
        itb.iternext()
    '''
    try:       
        Precall = n22 / ( n12 + n22)
        Pprecision = n22 / ( n21 + n22)
        f1_score = 2 / (1/Precall + 1/Pprecision)
        FP = n21/(n21 + n11)
        FN = n12/(n12 +n22 )
        BER = 1/2*(FP+FN)   
    except Exception as ex:
        print (ex.message)
    print ("n11:.."+str(n11)+"..n12:.."+str(n12)+"..n21:.."+str(n21)+"..n22:.."+str(n22))
    print ("TP:"+str(Precall))
    print ("f1 score:" + str(f1_score))
    print ("FP:"+ str(FP))
    print ("BER:" + str(BER))
    return Precall,f1_score,BER,FP

In [14]:
a = [1,2,3,4,5,6,7]
nlargest(3,a).key

AttributeError: 'list' object has no attribute 'key'

In [26]:
a =[1,2,3,4,5,6,7]
b = [10,122,189,1,587,6,11]
c = [a,b]
x = pd.DataFrame(data=c)
#t = nlargest(3,c[1][0])
t = nlargest(X_train['feature0'].all,3)
x

TypeError: 'int' object is not iterable

In [27]:
K=[4, 5, 1, 6, 2, 5, 2, 10]

In [32]:
np.argsort(K)[-2:]

array([3, 7], dtype=int64)